<a href="https://colab.research.google.com/github/imammarzuki/Eksperimen/blob/main/CBF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression

# Simulasi Data (Data asli harus dikumpulkan terlebih dahulu)
data = {
    'Usia': [25, 34, 28, 45, 33],
    'Jenis Kelamin': ['Laki-laki', 'Perempuan', 'Laki-laki', 'Perempuan', 'Laki-laki'],
    'Pendidikan': ['Sarjana', 'Diploma', 'SMA', 'Sarjana', 'Pascasarjana'],
    'Pekerjaan': ['PNS', 'Pekerja Kantoran', 'Wirausahawan', 'PNS', 'Pekerja Kantoran'],
    'Pendapatan': [7000000, 8000000, 5000000, 10000000, 9000000],
    'Bali': [4, 5, 4, 3, 5],
    'Yogyakarta': [5, 4, 3, 4, 5],
    'Lombok': [4, 5, 4, 2, 4],
    'Jakarta': [3, 4, 2, 3, 4],
    'Raja Ampat': [5, 5, 5, 4, 5]
}

df = pd.DataFrame(data)

# Preprocessing
le = LabelEncoder()
df['Jenis Kelamin'] = le.fit_transform(df['Jenis Kelamin'])
df['Pendidikan'] = le.fit_transform(df['Pendidikan'])
df['Pekerjaan'] = le.fit_transform(df['Pekerjaan'])

scaler = MinMaxScaler()
df[['Usia', 'Pendapatan']] = scaler.fit_transform(df[['Usia', 'Pendapatan']])

# Features and labels
features = df[['Usia', 'Jenis Kelamin', 'Pendidikan', 'Pekerjaan', 'Pendapatan']]
labels = df[['Bali', 'Yogyakarta', 'Lombok', 'Jakarta', 'Raja Ampat']]

# Content-Based Filtering
def predict_ratings(user_features, labels):
    predictions = []
    for i in range(labels.shape[1]):
        model = LinearRegression()
        model.fit(user_features, labels.iloc[:, i])
        predictions.append(model.predict(user_features))
    return np.array(predictions).T

predictions = predict_ratings(features, labels)
true_ratings = labels.values

# Evaluasi
mae = mean_absolute_error(true_ratings, predictions)
rmse = mean_squared_error(true_ratings, predictions, squared=False)

# Untuk metrik classification, kita perlu binarize ratings
threshold = 3.5
true_binary = (true_ratings > threshold).astype(int)
pred_binary = (predictions > threshold).astype(int)

accuracy = accuracy_score(true_binary.flatten(), pred_binary.flatten())
precision = precision_score(true_binary.flatten(), pred_binary.flatten(), average='weighted')
recall = recall_score(true_binary.flatten(), pred_binary.flatten(), average='weighted')
f1 = f1_score(true_binary.flatten(), pred_binary.flatten(), average='weighted')

# Output hasil evaluasi
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')


MAE: 1.9539925233402758e-16
RMSE: 3.3500223082034515e-16
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
